# Phase 3: GenAI Agent (RAG + LangGraph)

This notebook demonstrates the GenAI agent capabilities:
- Vector store for semantic search
- RAG pipeline for context-aware explanations
- LangGraph agent for autonomous risk analysis
- MCP-like tools for agent actions
- Privacy controls (PII detection/redaction)

In [ ]:
import sys
sys.path.insert(0, '..')

from datetime import datetime
from pprint import pprint

## 1. Vector Store for Semantic Search

The vector store uses Qdrant and sentence-transformers to enable semantic search over login events.

In [ ]:
from src.agents.vector_store import IdentityVectorStore, create_event_text

# Create in-memory vector store
vector_store = IdentityVectorStore()
print(f"Embedding dimension: {vector_store.embedding_dim}")
print(f"Model: {vector_store.embedding_model}")

In [ ]:
# Add sample events
sample_events = [
    {
        "event_id": "evt_001",
        "text": "Normal login from known device in San Francisco during business hours",
        "user_id": "employee_001",
        "risk_score": 0.1,
    },
    {
        "event_id": "evt_002",
        "text": "Suspicious login from unknown device in Russia with VPN, no MFA",
        "user_id": "user_002",
        "risk_score": 0.95,
    },
    {
        "event_id": "evt_003",
        "text": "Login from new mobile device at unusual hour (3am) but with MFA",
        "user_id": "employee_003",
        "risk_score": 0.4,
    },
    {
        "event_id": "evt_004",
        "text": "Multiple failed login attempts followed by successful login with VPN",
        "user_id": "user_004",
        "risk_score": 0.75,
    },
]

vector_store.add_events_batch(sample_events, tenant_id="demo")
print(f"Added {len(sample_events)} events to vector store")

In [ ]:
# Semantic search
results = vector_store.search(
    query="suspicious VPN login from unknown device",
    tenant_id="demo",
    limit=3,
)

print("Search results for 'suspicious VPN login from unknown device':")
for r in results:
    print(f"  Score: {r['score']:.3f} | {r['text'][:60]}...")

## 2. RAG Pipeline for Explanations

The RAG pipeline combines vector search with LLM generation (or template fallback) to explain risk decisions.

In [ ]:
from src.agents.rag import RAGPipeline

# Initialize RAG pipeline
rag = RAGPipeline()

# Ingest historical events
historical_events = [
    {
        "event_id": "hist_001",
        "user_id": "attacker_001",
        "device_id": "device_unknown_abc",
        "location_country": "RU",
        "mfa_used": False,
        "vpn_detected": True,
        "success": False,
        "risk_score": 0.92,
    },
    {
        "event_id": "hist_002",
        "user_id": "employee_002",
        "device_id": "laptop_002",
        "location_country": "US",
        "mfa_used": True,
        "vpn_detected": False,
        "success": True,
        "risk_score": 0.1,
    },
]

count = rag.ingest_events(historical_events, tenant_id="demo")
print(f"Ingested {count} historical events")

In [ ]:
# Get explanation for a suspicious event
explanation_result = rag.query(
    query="Why was this login flagged as high risk?",
    event={
        "user_id": "suspicious_user",
        "device_id": "device_unknown_xyz",
        "ip": "185.199.1.100",
        "location_country": "CN",
        "mfa_used": False,
        "vpn_detected": True,
    },
    risk_score=0.85,
    risk_level="high",
    risk_factors=["Unknown device", "VPN detected", "No MFA", "High-risk location"],
    tenant_id="demo",
)

print("Risk Explanation:")
print("=" * 50)
print(explanation_result["explanation"])
print(f"\nSimilar events found: {explanation_result['similar_events']}")

## 3. LangGraph Agent

The LangGraph agent autonomously investigates login events and recommends actions.

In [ ]:
from src.agents.agent import run_risk_agent, IdentityTools

# Test the agent with a normal login
print("=== Normal Login Analysis ===")
result = run_risk_agent(
    user_id="employee_001",
    device_id="laptop_001",
    ip="10.0.0.1",
    mfa_used=True,
    vpn_detected=False,
)

print(f"Risk Score: {result['risk_score']:.2f}")
print(f"Risk Level: {result['risk_level']}")
print(f"Recommended Action: {result['recommended_action']}")

In [ ]:
# Test the agent with a suspicious login
print("=== Suspicious Login Analysis ===")
result = run_risk_agent(
    user_id="unknown_user_999",
    device_id="device_unknown_xyz",
    ip="185.199.1.100",
    mfa_used=False,
    vpn_detected=True,
)

print(f"Risk Score: {result['risk_score']:.2f}")
print(f"Risk Level: {result['risk_level']}")
print(f"Recommended Action: {result['recommended_action']}")
print(f"\nInvestigation Notes:")
for note in result['investigation_notes']:
    print(f"  - {note}")

## 4. MCP-like Tools

The agent uses MCP-like tools with Pydantic schemas for structured input/output.

In [ ]:
from src.tools.risk_score import RiskScoreInput, execute as score_execute, TOOL_SCHEMA as risk_schema
from src.tools.user_history import UserHistoryInput, execute as history_execute, TOOL_SCHEMA as history_schema
from src.tools.quarantine import QuarantineInput, execute as quarantine_execute, TOOL_SCHEMA as quarantine_schema

# Show tool schemas
print("Available Tools:")
for schema in [risk_schema, history_schema, quarantine_schema]:
    print(f"  - {schema['name']}: {schema['description'][:50]}...")

In [ ]:
# Use risk score tool
risk_input = RiskScoreInput(
    user_id="user_001",
    device_id="device_unknown_123",
    ip="185.199.1.1",
    location_country="RU",
    mfa_used=False,
    vpn_detected=True,
)

risk_result = score_execute(risk_input)
print("Risk Score Tool Output:")
print(f"  Score: {risk_result.risk_score:.2f}")
print(f"  Level: {risk_result.risk_level}")
print(f"  Factors: {risk_result.factors}")

In [ ]:
# Use quarantine tool (dry run)
quarantine_input = QuarantineInput(
    user_id="suspicious_user",
    reason="High-risk login detected from unknown device with VPN",
    duration_hours=24,
    notify_user=True,
    notify_admin=True,
)

quarantine_result = quarantine_execute(quarantine_input, dry_run=True)
print("\nQuarantine Tool Output (Dry Run):")
print(f"  Action ID: {quarantine_result.action_id}")
print(f"  Status: {quarantine_result.status}")
print(f"  Expires: {quarantine_result.expires_at}")
print(f"  Notifications: {quarantine_result.notifications_sent}")
print(f"  Note: {quarantine_result.note}")

## 5. Privacy Controls (PII Detection)

Before sending data to LLMs, we detect and redact PII.

In [ ]:
from src.privacy.pii_detector import PIIDetector, PrivacyMiddleware

# Initialize PII detector
detector = PIIDetector()

# Test PII detection
test_texts = [
    "User john.doe@example.com logged in from 192.168.1.100",
    "Contact support at 555-123-4567 for help",
    "SSN: 123-45-6789 found in request",
    "Normal login from San Francisco at 3pm",
]

print("PII Detection Results:")
print("=" * 60)
for text in test_texts:
    result = detector.detect(text)
    print(f"\nOriginal: {text}")
    print(f"Redacted: {result.redacted_text}")
    print(f"PII Found: {result.has_pii} ({len(result.entities_found)} entities)")
    if result.entities_found:
        for e in result.entities_found:
            print(f"  - {e['type']}: '{e['text']}'")

In [ ]:
# Privacy middleware for LLM processing
middleware = PrivacyMiddleware()

processed = middleware.process_for_llm(
    text="User john.doe@example.com (SSN: 123-45-6789) logged in from IP 10.0.0.1",
    context={
        "email": "john.doe@example.com",
        "ip": "10.0.0.1",
        "user_id": "user_001",
    },
)

print("\nPrivacy Middleware Output:")
print(f"  Processed text: {processed['text']}")
print(f"  PII detected: {processed['pii_detected']}")
print(f"  Entity count: {processed['entity_count']}")
print(f"  Processed context: {processed['context']}")

## 6. End-to-End Demo

Complete flow: event -> agent analysis -> RAG explanation -> privacy-safe output.

In [ ]:
# Simulate incoming login event
incoming_event = {
    "user_id": "john.doe@example.com",  # Contains PII!
    "device_id": "device_unknown_new",
    "ip": "185.199.50.50",
    "location_country": "RU",
    "mfa_used": False,
    "vpn_detected": True,
}

print("=" * 60)
print("END-TO-END RISK ANALYSIS")
print("=" * 60)

# 1. Run agent analysis
print("\n1. Agent Analysis:")
agent_result = run_risk_agent(
    user_id=incoming_event["user_id"],
    device_id=incoming_event["device_id"],
    ip=incoming_event["ip"],
    mfa_used=incoming_event["mfa_used"],
    vpn_detected=incoming_event["vpn_detected"],
)
print(f"   Risk Score: {agent_result['risk_score']:.2f}")
print(f"   Risk Level: {agent_result['risk_level'].upper()}")
print(f"   Action: {agent_result['recommended_action']}")

# 2. Get RAG explanation
print("\n2. RAG Explanation:")
explanation = rag.query(
    query="Why is this login suspicious?",
    event=incoming_event,
    risk_score=agent_result["risk_score"],
    risk_level=agent_result["risk_level"],
    risk_factors=["Unknown device", "VPN", "No MFA", "High-risk country"],
)
print(explanation["explanation"][:500] + "...")

# 3. Apply privacy controls before logging
print("\n3. Privacy-Safe Log Entry:")
safe_log = middleware.process_for_llm(
    text=f"Risk alert for {incoming_event['user_id']} from IP {incoming_event['ip']}",
    context=incoming_event,
)
print(f"   {safe_log['text']}")
print(f"   PII entities redacted: {safe_log['entity_count']}")

## Summary

Phase 3 implements:

1. **Vector Store** - Qdrant + sentence-transformers for semantic event search
2. **RAG Pipeline** - Context-aware explanations combining retrieval + generation
3. **LangGraph Agent** - Autonomous risk analysis with state machine workflow
4. **MCP-like Tools** - Pydantic-validated tools for agent actions
5. **Privacy Controls** - PII detection and redaction before LLM processing

The agent can:
- Analyze login events and compute risk scores
- Investigate high-risk events by retrieving user history
- Search for similar historical events
- Recommend actions (block, step-up auth, monitor)
- Generate human-readable explanations
- Protect PII throughout the pipeline